In [12]:
import random
from skimage import io
import matplotlib.pyplot as plt
from ipywidgets import interact
from keras.models import Model, Sequential 
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
import numpy as np
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D, Merge, UpSampling2D
from keras.layers import Reshape
from keras.layers.normalization import BatchNormalization
from keras.losses import mse, binary_crossentropy
from keras.optimizers import RMSprop
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.layers.core import Lambda
import keras
import os
from sklearn.metrics import accuracy_score
import seaborn as sns
from keras.utils.vis_utils import plot_model
import pandas as pd
import sys
sys.path.append("../")
from networks.networks import *
import h5py
from utils.load_data import *
from utils.utils import *

In [4]:
TRAIN_CSV = '../datasets/dataset2/train_pairs.csv'
TEST_CSV = '../datasets/dataset2/test_pairs.csv'

In [5]:
data = load_data(TRAIN_CSV, TEST_CSV)

[X1, X2] = data["X"]
y = data["y"]
[X1_fn, X2_fn] = data["X_fn"]

[X1_test, X2_test] = data["X_test"]
y_test = data["y_test"]
[X1_test_fn, X2_test_fn] = data["X_test_fn"]


In [7]:
encoder = load_model("../models/encoder512.h5")

/home/jchan/anaconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
X1 = X1[:, 3:-3, 1:, :]
X2 = X2[:, 3:-3, 1:, :]

X1_test = X1_test[:, 3:-3, 1:, :]
X2_test = X2_test[:, 3:-3, 1:, :]

In [10]:
print("Predicting Training Data with ...")
train_pred_x1 = encoder.predict(X1)
train_pred_x2 = encoder.predict(X2)

print("Predicting Testing Data with ...")
test_pred_x1 = encoder.predict(X1_test)
test_pred_x2 = encoder.predict(X2_test)

Predicting Training Data with ...
Predicting Testing Data with ...


In [23]:
train_dist = distance(np.array(train_pred_x1)[2], np.array(train_pred_x2)[2])
test_dist = distance(np.array(test_pred_x1)[2], np.array(test_pred_x2)[2]) 

In [24]:
CURRENT_OUTPUT_PATH = "../stats/encoder512"

df_dict_train = {
    "X1": X1_fn,
    "X2": X2_fn,
    "y_pred": train_dist,
    "y_true": y
}

df_dict_test = {
    "X1": X1_test_fn,
    "X2": X2_test_fn,
    "y_pred": test_dist,
    "y_true": y_test
}

train_csv_filename = CURRENT_OUTPUT_PATH + "/{0}-train_results.csv".format("encoder512")
test_csv_filename = CURRENT_OUTPUT_PATH + "/{0}-test_results.csv".format("encoder512")

In [26]:
train_df = pd.DataFrame.from_dict(df_dict_train)
train_df["training"] = 1
print("Saving train csv in {}".format(train_csv_filename))
train_df.to_csv(train_csv_filename, index=False)

test_df = pd.DataFrame.from_dict(df_dict_test)
test_df["training"] = 0
print("Saving test csv in {}".format(test_csv_filename))
test_df.to_csv(test_csv_filename, index=False)

Saving train csv in ../stats/encoder512/encoder512-train_results.csv
Saving test csv in ../stats/encoder512/encoder512-test_results.csv
